# Segmenting and Clustering Neighborhoods in Toronto

# Part 1: Segment Neighborhoods into a Dataframe

#### Import Libraries

In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup as bs4
import lxml
import requests
#!conda install -c conda-forge geopy --yes
#from geopy.geocoders import Nominatim 
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library
print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge
Libraries imported.


#### Import Wikipedia Article (Part 2)

In [2]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = bs4(source, 'lxml')

#### Define the header columns (Part 3, Bullet 1)

In [3]:
header = ['PostalCode', 'Borough', 'Neighborhood']
#If pulling from wikipedia article then:
#header = soup.tr.text
#header = header.split()
header

['PostalCode', 'Borough', 'Neighborhood']

#### Pull the records from the table in Wikipedia (Part 2)

In [4]:
l_records = []
table_rows = soup.find_all('tr')
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    l_records.append(row)
print("First 3 rows: ")
print(l_records[:3])
print("Last 3 rows: ")
print(l_records[-3:])

First 3 rows: 
[[], ['M1A', 'Not assigned', 'Not assigned\n'], ['M2A', 'Not assigned', 'Not assigned\n']]
Last 3 rows: 
[['\n\n\nNL\n\nNS\n\nPE\n\nNB\n\nQC\n\nON\n\nMB\n\nSK\n\nAB\n\nBC\n\nNU/NT\n\nYT\n\n\nA\n\nB\n\nC\n\nE\n\nG\n\nH\n\nJ\n\nK\n\nL\n\nM\n\nN\n\nP\n\nR\n\nS\n\nT\n\nV\n\nX\n\nY\n\n', 'NL\n', 'NS\n', 'PE\n', 'NB\n', 'QC\n', 'ON\n', 'MB\n', 'SK\n', 'AB\n', 'BC\n', 'NU/NT\n', 'YT\n', 'A\n', 'B\n', 'C\n', 'E\n', 'G\n', 'H\n', 'J\n', 'K\n', 'L\n', 'M\n', 'N\n', 'P\n', 'R\n', 'S\n', 'T\n', 'V\n', 'X\n', 'Y\n'], ['NL\n', 'NS\n', 'PE\n', 'NB\n', 'QC\n', 'ON\n', 'MB\n', 'SK\n', 'AB\n', 'BC\n', 'NU/NT\n', 'YT\n'], ['A\n', 'B\n', 'C\n', 'E\n', 'G\n', 'H\n', 'J\n', 'K\n', 'L\n', 'M\n', 'N\n', 'P\n', 'R\n', 'S\n', 'T\n', 'V\n', 'X\n', 'Y\n']]


#### Remove unnecessary records pulled above.

In [5]:
l_list = l_records[1:-5]
l_list[:5]

[['M1A', 'Not assigned', 'Not assigned\n'],
 ['M2A', 'Not assigned', 'Not assigned\n'],
 ['M3A', 'North York', 'Parkwoods\n'],
 ['M4A', 'North York', 'Victoria Village\n'],
 ['M5A', 'Downtown Toronto', 'Harbourfront\n']]

#### Create DataFrame from Data (Part 3 Start)

In [6]:
df_list = pd.DataFrame(l_list, columns = header)
df_list.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


In [7]:
df_list['Neighborhood'] = df_list['Neighborhood'].map(lambda x: str(x)[:-1])
print("DataFrame has ", len(df_list), "records")
df_list.head()

DataFrame has  289 records


,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#### Remove Records with 'Not Assigned' as Borough (Part 3, Bullet 2)

In [8]:
df_wB = df_list[df_list.Borough != 'Not assigned']
#wB = with Borough
print("DataFrame has ", len(df_wB), " records")
df_wB.head()

DataFrame has  212  records


,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


#### Combine Neighborhoods to Postal Code (Part 3, Bullet 3)

In [9]:
df_pc = df_wB
df_pc2 = df_pc

#### Grouping Neighborhoods by Postal Code

In [10]:
df_pc3 = df_pc2.groupby('PostalCode')['Neighborhood'].apply(lambda tags: ', '.join(tags))
df_pc3.head()

PostalCode
M1B                            Rouge, Malvern
M1C    Highland Creek, Rouge Hill, Port Union
M1E         Guildwood, Morningside, West Hill
M1G                                    Woburn
M1H                                 Cedarbrae
Name: Neighborhood, dtype: object

#### Making the series back into a dataframe

In [11]:
df_n = pd.DataFrame({'PostalCode':df_pc3.index, 'Neighborhood':df_pc3.values})
df_n.head()

,Neighborhood,PostalCode
0,"Rouge, Malvern",M1B
1,"Highland Creek, Rouge Hill, Port Union",M1C
2,"Guildwood, Morningside, West Hill",M1E
3,Woburn,M1G
4,Cedarbrae,M1H


#### Combining the two dataframes with the new Neighborhood format

In [12]:
df_combined = pd.DataFrame(df_pc,columns=['PostalCode', 'Borough'])
df_combined.head()

,PostalCode,Borough
2,M3A,North York
3,M4A,North York
4,M5A,Downtown Toronto
5,M5A,Downtown Toronto
6,M6A,North York


In [13]:
df_combined = df_combined.drop_duplicates()
df_combined.head()

,PostalCode,Borough
2,M3A,North York
3,M4A,North York
4,M5A,Downtown Toronto
6,M6A,North York
8,M7A,Queen's Park


In [14]:
df_n.head()

,Neighborhood,PostalCode
0,"Rouge, Malvern",M1B
1,"Highland Creek, Rouge Hill, Port Union",M1C
2,"Guildwood, Morningside, West Hill",M1E
3,Woburn,M1G
4,Cedarbrae,M1H


In [15]:
df = pd.merge(df_combined, df_n, on='PostalCode')

In [16]:
print("DataFrame has ", len(df), " records")
df.head()

DataFrame has  103  records


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Not assigned


In [17]:
df.shape

(103, 3)

# Part 2: Bringing in Longitude & Latitude

#### Import CSV

In [18]:
geo_data = pd.read_csv('http://cocl.us/Geospatial_data')
geo_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Joing Lat & Long to previous Dataframe

In [19]:
df_geo = pd.merge(df, geo_data, left_on='PostalCode', right_on='Postal Code')
df_geo = df_geo.drop(['Postal Code'], axis=1)
df_geo.head(103)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Not assigned,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


In [20]:
df_geo.shape

(103, 5)

# Part 3: Clustering Neighborhoods into Map

In [21]:
#Below was used to gather data. After running, redfined as text
#address = 'Toronto, ON'
#geolocator = Nominatim()
#location = geolocator.geocode(address)
#latitude = location.latitude
#longitude = location.longitude
#print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

latitude = 43.653963
longitude = -79.387207
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


#### Create a map of Toronto 

In [22]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_geo['Latitude'], df_geo['Longitude'], df_geo['Borough'], df_geo['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='purple',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [23]:
toronto_data = df_geo[df_geo['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
1,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


# FourSquare Analysis for Toronto (Similar to NYC lab)

In [24]:
CLIENT_ID = 'LPBLOG04ODIT2QMCE4TEJJ2UJA1HVKMWHO2BSV212GOVKTTN' # your Foursquare ID
CLIENT_SECRET = 'JUIXKQ0T2L11UEQ3REAF2G0KFTAMRD14Y21ODSPYYRLMKZYU' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: LPBLOG04ODIT2QMCE4TEJJ2UJA1HVKMWHO2BSV212GOVKTTN
CLIENT_SECRET:JUIXKQ0T2L11UEQ3REAF2G0KFTAMRD14Y21ODSPYYRLMKZYU


In [25]:
toronto_data.loc[0, 'Neighborhood']  

'Harbourfront, Regent Park'

In [26]:
#Selection of a borough
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[0, 'Neighborhood']

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Harbourfront, Regent Park are 43.6542599, -79.3606359.


#### Retrieving Venues in Toronoto through FourSquare.  Export in json files translated into a DataFrame.

In [27]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

In [28]:
results = requests.get(url).json()

In [29]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [30]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Toronto Cooper Koo Family Cherry St YMCA Centre,Gym / Fitness Center,43.653191,-79.357947
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149


In [31]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

46 venues were returned by Foursquare.


#### Exporting Neighborhood Groupings in Toronto

In [32]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [33]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'], latitudes=toronto_data['Latitude'], longitudes=toronto_data['Longitude'])

Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide, King, Richmond
Dovercourt Village, Dufferin
Harbourfront East, Toronto Islands, Union Station
Little Portugal, Trinity
The Danforth West, Riverdale
Design Exchange, Toronto Dominion Centre
Brockton, Exhibition Place, Parkdale Village
The Beaches West, India Bazaar
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North, Forest Hill West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
Harbord, University of Toronto
Runnymede, Swansea
Moore Park, Summerhill East
Chinatown, Grange Park, Kensington Market
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown, St. James Town
Fir

In [34]:
print(toronto_venues.shape)
toronto_venues.head()

(1700, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Harbourfront, Regent Park",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Harbourfront, Regent Park",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Harbourfront, Regent Park",43.65426,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,"Harbourfront, Regent Park",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Harbourfront, Regent Park",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


In [35]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,58,58,58,58,58,58
"Brockton, Exhibition Place, Parkdale Village",21,21,21,21,21,21
Business Reply Mail Processing Centre 969 Eastern,17,17,17,17,17,17
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",13,13,13,13,13,13
"Cabbagetown, St. James Town",46,46,46,46,46,46
Central Bay Street,80,80,80,80,80,80
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
Christie,16,16,16,16,16,16


In [36]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 238 uniques categories.


#### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [37]:
#create new dataframe

# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,"Adelaide, King, Richmond",0.000000,0.01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.00,0.010000,0.000000,0.000000,0.010000,0.000000,0.01
1,Berczy Park,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.00,0.017241,0.000000,0.000000,0.000000,0.000000,0.00
2,"Brockton, Exhibition Place, Parkdale Village",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
3,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.000000,0.00,0.000000,0.076923,0.076923,0.076923,0.153846,0.153846,0.153846,...,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
5,"Cabbagetown, St. James Town",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
6,Central Bay Street,0.012500,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.00,0.012500,0.000000,0.000000,0.012500,0.000000,0.00
7,"Chinatown, Grange Park, Kensington Market",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.01,0.000000,0.000000,0.00,0.050000,0.000000,0.040000,0.010000,0.000000,0.00
8,Christie,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
9,Church and Wellesley,0.011765,0.00,0.011765,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.00,0.000000,0.011765,0.011765,0.000000,0.011765,0.00


In [39]:
toronto_grouped.shape

(38, 238)

In [40]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
             venue  freq
0      Coffee Shop  0.06
1             Café  0.04
2       Steakhouse  0.04
3              Bar  0.04
4  Thai Restaurant  0.04


----Berczy Park----
          venue  freq
0   Coffee Shop  0.09
1  Cocktail Bar  0.05
2           Pub  0.03
3          Café  0.03
4   Cheese Shop  0.03


----Brockton, Exhibition Place, Parkdale Village----
            venue  freq
0  Breakfast Spot  0.10
1     Coffee Shop  0.10
2            Café  0.10
3    Climbing Gym  0.05
4             Bar  0.05


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0  Light Rail Station  0.12
1          Comic Shop  0.06
2          Restaurant  0.06
3          Skate Park  0.06
4                 Spa  0.06


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
              venue  freq
0    Airport Lounge  0.15
1   Airport Service  0.15
2  Airport Terminal  0.15
3          Bout

In [41]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [42]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Steakhouse,Café,Thai Restaurant,Bar,Burger Joint,Gym,Sushi Restaurant,Bakery,Hotel
1,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Cheese Shop,Bakery,Steakhouse,Farmers Market,Pub,Café,Restaurant
2,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Breakfast Spot,Café,Pet Store,Burrito Place,Restaurant,Caribbean Restaurant,Climbing Gym,Gym,Bar
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Auto Workshop,Garden,Smoke Shop,Brewery,Farmers Market,Fast Food Restaurant,Spa,Burrito Place,Restaurant
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Terminal,Airport Service,Harbor / Marina,Sculpture Garden,Boutique,Boat or Ferry,Airport Gate,Airport,Airport Food Court
5,"Cabbagetown, St. James Town",Coffee Shop,Restaurant,Pizza Place,Park,Italian Restaurant,Bakery,Café,Pub,Breakfast Spot,Plaza
6,Central Bay Street,Coffee Shop,Italian Restaurant,Bubble Tea Shop,Burger Joint,Bar,Café,Chinese Restaurant,Thai Restaurant,Salad Place,Spa
7,"Chinatown, Grange Park, Kensington Market",Café,Bar,Vegetarian / Vegan Restaurant,Chinese Restaurant,Bakery,Vietnamese Restaurant,Coffee Shop,Caribbean Restaurant,Arts & Crafts Store,Burger Joint
8,Christie,Grocery Store,Café,Park,Italian Restaurant,Athletics & Sports,Restaurant,Diner,Nightclub,Coffee Shop,Convenience Store
9,Church and Wellesley,Coffee Shop,Japanese Restaurant,Restaurant,Sushi Restaurant,Gay Bar,Burger Joint,Bubble Tea Shop,Café,Men's Store,Mediterranean Restaurant


# Clustering Toronto

In [43]:
kclusters = 6
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [44]:
toronto_merged = toronto_data

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() 

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,1,Coffee Shop,Bakery,Café,Pub,Park,Breakfast Spot,Restaurant,Theater,Mexican Restaurant,French Restaurant
1,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,1,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Tea Room,Japanese Restaurant,Italian Restaurant,Bubble Tea Shop,Pizza Place
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Coffee Shop,Restaurant,Café,Hotel,Breakfast Spot,Clothing Store,Cosmetics Shop,Bakery,Gastropub,BBQ Joint
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Health Food Store,Coffee Shop,Pub,Dim Sum Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,1,Coffee Shop,Cocktail Bar,Seafood Restaurant,Cheese Shop,Bakery,Steakhouse,Farmers Market,Pub,Café,Restaurant


#### Visualizing the Clusters 

In [45]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Examine Clusters

<b>Cluster 1</b>

In [53]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1]+[2] + list(range(6, toronto_merged.shape[1]))]]

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,West Toronto,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Breakfast Spot,Café,Pet Store,Burrito Place,Restaurant,Caribbean Restaurant,Climbing Gym,Gym,Bar
35,Downtown Toronto,"First Canadian Place, Underground city",Coffee Shop,Café,Hotel,Steakhouse,Bakery,American Restaurant,Deli / Bodega,Burger Joint,Bar,Restaurant


Observation: Very similar pattern from coffee cafes, speciality shops and bars.

<b>Cluster 2</b>

In [54]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1]+[2] + list(range(6, toronto_merged.shape[1]))]]

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,"Harbourfront, Regent Park",Coffee Shop,Bakery,Café,Pub,Park,Breakfast Spot,Restaurant,Theater,Mexican Restaurant,French Restaurant
1,Downtown Toronto,"Ryerson, Garden District",Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Tea Room,Japanese Restaurant,Italian Restaurant,Bubble Tea Shop,Pizza Place
2,Downtown Toronto,St. James Town,Coffee Shop,Restaurant,Café,Hotel,Breakfast Spot,Clothing Store,Cosmetics Shop,Bakery,Gastropub,BBQ Joint
3,East Toronto,The Beaches,Health Food Store,Coffee Shop,Pub,Dim Sum Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
4,Downtown Toronto,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Cheese Shop,Bakery,Steakhouse,Farmers Market,Pub,Café,Restaurant
5,Downtown Toronto,Central Bay Street,Coffee Shop,Italian Restaurant,Bubble Tea Shop,Burger Joint,Bar,Café,Chinese Restaurant,Thai Restaurant,Salad Place,Spa
6,Downtown Toronto,Christie,Grocery Store,Café,Park,Italian Restaurant,Athletics & Sports,Restaurant,Diner,Nightclub,Coffee Shop,Convenience Store
7,Downtown Toronto,"Adelaide, King, Richmond",Coffee Shop,Steakhouse,Café,Thai Restaurant,Bar,Burger Joint,Gym,Sushi Restaurant,Bakery,Hotel
8,West Toronto,"Dovercourt Village, Dufferin",Discount Store,Bakery,Supermarket,Pharmacy,Middle Eastern Restaurant,Furniture / Home Store,Bus Stop,Café,Bar,Bank
9,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",Coffee Shop,Hotel,Aquarium,Italian Restaurant,Café,Pizza Place,Brewery,Scenic Lookout,Fried Chicken Joint,Restaurant


Observation: Lots of cafes then shops and bars.

<B>Cluster 3</b>

In [56]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1]+[2] + list(range(6, toronto_merged.shape[1]))]]

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
24,West Toronto,"Parkdale, Roncesvalles",Breakfast Spot,Gift Shop,Dog Run,Movie Theater,Burger Joint,Eastern European Restaurant,Bar,Bank,Restaurant,Italian Restaurant
27,West Toronto,"Runnymede, Swansea",Café,Coffee Shop,Pizza Place,Italian Restaurant,Sushi Restaurant,Gourmet Shop,Spa,Indie Movie Theater,Dessert Shop,Post Office


Observation: Quick food first, then specialization shops and restaurants.

<B>Cluster 4</b>

In [57]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1]+[2] + list(range(6, toronto_merged.shape[1]))]]

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Central Toronto,North Toronto West,Clothing Store,Coffee Shop,Sporting Goods Shop,Ice Cream Shop,Fast Food Restaurant,Diner,Metro Station,Mexican Restaurant,Dessert Shop,Park


Observation: Its own cluster because of clothing store as first venue then restuarants

<b>Cluster 5</b>

In [58]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1]+[2] + list(range(6, toronto_merged.shape[1]))]]

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Central Toronto,Lawrence Park,Park,Swim School,Bus Line,Fast Food Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


Observation: Its own cluster due to physical venues at forefront then restaurants

<b>Cluster 6</b>

In [59]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, toronto_merged.columns[[1]+[2] + list(range(6, toronto_merged.shape[1]))]]

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,Central Toronto,"Moore Park, Summerhill East",Gym,Park,Playground,Women's Store,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


Observation: Its own cluster due to physical venues at forefront (gym, park, playground) then clothing